<a href="https://colab.research.google.com/github/gloriaconcepto/ML-NLP-LEARNING/blob/main/loneliness_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CREATE A SAMPLE MODEL THAT WILL BE USED FOR BASICS PREDICTION(REDDITS LONELINESS PROJECT)**

In [1]:
# mount your google drive
#if your data is store on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#install transformer
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00


In [3]:
#import neccessary libraries
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
from tqdm import tqdm

In [4]:
#Load a large CSV file in chunks
chunk_size = 100

#please your file path would be difference from mind
data_file_path = '/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/small-loneliness-dataset.csv'

column_names = ['message', 'label']

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Initialize the BERT model for sequence classification
#num_labels really matter
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Define your optimizer
optimizer = AdamW(model.parameters(), lr=0.0001)



(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


***Training ***

In [5]:
from sklearn.model_selection import train_test_split
epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for epoch in range(epochs):
    # Read the CSV file in chunks
     for chunk in tqdm(pd.read_csv(data_file_path, chunksize=chunk_size, names=column_names), desc=f"Epoch {epoch+1}/{epochs}"):
        # Process each chunk
        texts = chunk['message'].tolist()
        labels = chunk['label'].tolist()

        # Tokenize the input texts using BERT tokenizer
        tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

        # Convert labels to a PyTorch tensor
        int_labels = [0]
        for i in range(1,len(labels)):
             int_labels.append(i)

        labels = torch.tensor(int_labels)

        # Combine tokenized texts and labels into a TensorDataset
        dataset = TensorDataset(tokenized_texts['input_ids'], tokenized_texts['attention_mask'], labels)

        # Split the dataset into training and validation sets
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        print(train_size)

        # Create DataLoader instances for training and validation
        train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

        # Training loop for the chunk
        model.train()
        for batch in tqdm(train_dataloader, desc="Training"):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels.float())
            loss = outputs.loss
            loss.backward()
            optimizer.step()



        # Validation for the chunk
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                input_ids, attention_mask, labels = batch
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels.float())
                total_val_loss += outputs.loss.item()

        average_val_loss = total_val_loss / len(val_dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {average_val_loss}")

# Your model is now trained!
torch.save(model.state_dict(), '/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/model.pth')

model.save_pretrained('/content/drive/MyDrive/HowardNLPResearch/reddit-loneliness-project/model2.pth')

Epoch 1/3: 0it [00:00, ?it/s]

400



Training: 100%|██████████| 50/50 [42:09<00:00, 50.60s/it]

Validation: 100%|██████████| 13/13 [03:14<00:00, 14.99s/it]
Epoch 1/3: 1it [45:25, 2725.47s/it]

Epoch 1/3, Validation Loss: 81323.83804086539
400



Training: 100%|██████████| 50/50 [31:34<00:00, 37.88s/it]

Validation: 100%|██████████| 13/13 [02:26<00:00, 11.31s/it]
Epoch 1/3: 2it [1:19:27, 2323.66s/it]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py:414: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
Epoch 1/3: 2it [1:19:27, 2383.94s/it]


Epoch 1/3, Validation Loss: 81138.48287259616
0


ValueError: ignored